Install and import necessary libraries

In [ ]:
!pip install lxml
!pip install serpapi

In [168]:
import pandas as pd
import json
import re
from bs4 import BeautifulSoup
import requests
import time

from urllib.parse import urlencode, urlunparse
from urllib.request import urlopen, Request

Load stratups JSON data into a dataframe

In [115]:
with open('') as f: #startups_json.txt
    json_data = json.load(f)

In [116]:
data = json_data['data']
startups_list = data['startups']

In [ ]:
for obj in startups_list:
    for val in obj:
        #print(obj[val])
        if(str(val)!='website'):
            #print(str(val))
            obj[val] = re.sub('<[^<]+?>', '', str(obj[val]))
        elif(str(val)=='website'):
            #obj[val] = re.sub('\\\\', '', str(obj[val]))
            #print(obj[val])
            soup = BeautifulSoup(str(obj[val]),"html.parser")
            tag = soup.find('a', href=True)
            obj[val] = str(tag['href'])

In [118]:
df = pd.DataFrame.from_dict(startups_list, orient='columns')

Functions to search and extract urls based on company name and website

In [221]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/111.0.5563.65 Safari/537.36"}

def extract_elements(html_text, elm='p'):
    soup = BeautifulSoup(html_text, 'html.parser')
    paragraphs = []
    for p in soup.find_all(elm):
        paragraphs.append(p.text.strip())
    return paragraphs

def google_search_alt(q: str):
    time.sleep(3)
    params = {'q': q}
    #html = requests.get('https://www.google.com/search', headers=headers, params=params).text
    html = requests.get('https://www.bing.com/search', headers=headers, params=params).text
    soup = BeautifulSoup(html, "html.parser")
    elms = extract_elements(str(soup), elm='cite')
    #print(elms)
    return elms

Form search query term and add to a new column

In [120]:
name_website_list = []
for index in range(len(df)):
    name_website_list.append(df['startup'].iloc[index] +' '+ df['website'].iloc[index]+' linkedin about')

In [121]:
df['name_website_list'] = name_website_list

Loop through dataframe and extract url list from the search results

In [ ]:
linkedin_list = []
i = 0
for index in range(len(df)):
    i = i +1
    website = df['name_website_list'].iloc[index]
    url = google_search_alt(q=website)
    if(len(url)>0):
        linkedin_list.append(url)
    else:
        linkedin_list.append([])

Create a new column with the extracted data and save file as csv

In [228]:
df['linkedin_list'] = linkedin_list

In [227]:
df.to_csv('linked_in_scrape_v2', sep='\t', encoding='utf-8')